# GridWorld Reinforcement Learning

Reinforcement Learning implementation of GridWorld in [Reinforcement Learning: An Introduction, p72](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf) and [Markov Decision Process and Exact Solution Methods](https://people.eecs.berkeley.edu/~pabbeel/cs287-fa19/slides/Lec2-mdps-exact-methods.pdf). The following descibes the environment and the agent:

- The agent (robot) lives in a grid
- Walls block the agent’s path
- The agent’s actions do not always go as planned:
    - 80% of the time, the action North takes the agent North (if there is no wall there)
    - 10% of the time, North takes the agent West; 10% East
    - If there is a wall in the direction the agent would have been taken, the agent stays put

## `GridWorld` Class

The `GridWorld` class solves both the model-based and model-free version of the problem.
- Normal states are represented as coordinates *i.e.* $(x,y,0)$.
- Goal states are flagged by its third coordinate *i.e.* $(x,y,1)$.
- `GridWorld` has a `step` method to let the agent perform an action. 
- `GridWorld` has a `render` method to render the environment and agent.
- For the model-based version of the problem, it can perform `ValueIteration` and `PolicyIteration` to determine the optimal policy.
- For the model-free version of the problem, it uses `keras-rl`, `tensorflow`, and `gym` to determine the optimal policy.

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import cv2

In [2]:
class GridWorld(Env):
  # Action space
  actions = [0, 1, 2, 3]  # up, right, down, left
  convert = {0: "↑", 1: "→", 2: "↓", 3: "←", ".": "."}

  # Agent rendering
  scale = 100
  agent = cv2.resize(cv2.imread("assets/Robot.jpg")/255, (scale, scale))
  
  def __init__(self, size, start = [0,0,0], noise = 0.2, reward = 0, gamma = 0.9):
    # Open AI Gym parameters
    self.action_space = Discrete(4)
    self.observation_space = Box(low = 0, high = max(size), shape=(1, 3))
    self.length = 1000
    
    # Environment and State parameters
    self.blocked = []
    self.nterminal = []
    self.size = size
    self.N = self.size[0] * self.size[1]
    self.state = start
    self.start = start

    # Other parameters
    self.noise = noise
    self.reward_map = {i:reward for i in range(self.N)}
    self.gamma = gamma

    # MDP initialization
    self.value = [0 for _ in range(2*self.N)]
    self.policy = [['.'] for _ in range(self.N)]
    self.Q = [[0 for _ in range(len(self.actions))] for _ in range(self.N)]
    
    # Canvas initialization
    self.canvas = np.ones((self.size[1] * self.scale, self.size[0] * self.scale, 3)) * 1
    self.bumped = -1
    self.oops = ""

  # Converting from integers to coordinate systems
  def to_state(self, coord):
    return self.size[0] * coord[1] + coord[0] + self.N * coord[2] 
  def to_coord(self, state):
    return [(state%self.N)%self.size[0], (state%self.N)//self.size[0], state//self.N]
  
  # Add a reward for a specific state
  def add_reward(self, coord, val):
    self.reward_map[self.to_state(coord)] = val

  # Add blocked states
  def add_blocked(self, coord):
    self.blocked.append(self.to_state(coord))

    x = coord[0]; y = self.size[1] - coord[1]
    self.canvas[y*self.scale-self.scale : y*self.scale, x*self.scale : x*self.scale+self.scale] = 0
  
  # Add near terminal states (penultimate states from goal states)
  def add_nterminal(self, coord, val):
    self.nterminal.append(self.to_state(coord))
    goal_coord = coord.copy()
    goal_coord[2] = 1
    self.add_reward(goal_coord, val)

    x = coord[0]; y = self.size[1] - coord[1]
    adj = 1 - abs(val) / 10
    self.canvas[y*self.scale-self.scale : y*self.scale, x*self.scale : x*self.scale+self.scale] = [adj,1,adj] if val > 0 else [adj,adj,1]
      
  # Method to let the agent perform an action
  def step(self, action, noise = None, state = None):
    curr_state = self.state if state is None else state.copy()
    curr_noise = self.noise if noise is None else noise
    self.length -= 0 if state is None else 1

    # Check terminal states
    if curr_state[2] == 1:
      return curr_state, self.reward_map[self.to_state(curr_state)], True, {}
    
    # Check near terminal states
    if self.to_state(curr_state) in self.nterminal:
      curr_state[2] = 1
      return curr_state, self.reward_map[self.to_state(curr_state)], True, {}

    # Add noise
    p = np.random.uniform()
    if p < curr_noise / 2:
      action = (action + 1 + 4) % 4
      self.oops = "Oops..."
    elif p < curr_noise:
      action = (action - 1 + 4) % 4
      self.oops = "Oops..."
    else:
      self.oops = ""

    # Move to next state
    if action == 0:   # up
      curr_state = [curr_state[0], curr_state[1] + 1, curr_state[2]]
    elif action == 1: # right
      curr_state = [curr_state[0] + 1, curr_state[1], curr_state[2]]
    elif action == 2: # down
      curr_state = [curr_state[0], curr_state[1] - 1, curr_state[2]]
    elif action == 3: # left
      curr_state = [curr_state[0] - 1, curr_state[1], curr_state[2]]
    
    # Check walls and blocked
    if self.to_state(curr_state) not in self.blocked:
      walled_state = [min(self.size[0]-1,max(curr_state[0],0)), 
                    min(self.size[1]-1,max(curr_state[1],0)),
                    curr_state[2]]
      if curr_state != walled_state:
        self.bumped = action
        curr_state = walled_state
      else:
        self.bumped = -1
    else:
      curr_state = self.state if state is None else state
      self.bumped = action

    # Calculate reward
    reward = self.reward_map[self.to_state(curr_state)] 

    # Check if is done
    done = self.length == 0

    # Set placeholder for info
    info = {}
    
    # Update self.state
    if state is None:
      self.state = curr_state

    # Return step information
    return curr_state, reward, done, info
  
  # Value Iteration to determine the optimal policy
  def ValueIteration(self, max_iter = 1000, tolerance = 1e-5, show_iter = False):

    for it in range(max_iter):
      new_value = self.value.copy()   # Create new copy
      delta = 0                       # Max across all states

      # Check all states
      for state in range(self.N):
        if state in self.blocked:   # Skip blocked states
          continue

        # Current coordinate
        curr_coord = self.to_coord(state)
        max_value = -1e9

        # Run through all actions
        for action in self.actions:
          self.Q[state][action] = 0

          # Check all possible next states
          for action_next in self.actions:
            
            # (1 - noise) if correct next state
            if action_next == action:
              prob = 1 - self.noise

            # noise / 2 if adjacent next state
            elif action_next != (action + 2) % 4:
              prob = self.noise / 2
            
            # 0 if opposite next state
            else:
              prob = 0

            state_next, reward, done, info = self.step(action_next, noise=0, state=curr_coord)
            state_next = self.to_state(state_next)
            
            # Get sum 
            self.Q[state][action] += prob * (reward + self.gamma * self.value[state_next])
            
          # Get max across all actions
          max_value = max(max_value, self.Q[state][action])

        # Update max value across all states
        delta = max(delta, abs(max_value - new_value[state]))

        # Update current state
        new_value[state] = max_value

      # Replace variable
      self.value = new_value

      # Break if threshold is reached
      if delta < tolerance or it == max_iter - 1:
        print("Final")
        self.OptimalPolicy()
        self.show_current(policy=True)
        break

      # Display iteration
      if show_iter:
        print(f"Iteration: {it+1}")
        self.OptimalPolicy()
        self.show_current()

  # Find Optimal Policy
  def OptimalPolicy(self):
    for state in range(self.N):
      if state in self.nterminal + self.blocked:   # Skip nterminal and blocked states
        continue
      self.policy[state] = list(np.flatnonzero(self.Q[state] == np.max(self.Q[state])))

  # Policy Evaluation
  def PolicyEvaluation(self, max_iter = 100, tolerance = 1e-5):
    for it in range(max_iter):
      new_value = self.value.copy()     # Create new copy
      delta = 0                         # Max across all states

      # Check all states
      for state in range(self.N):
        curr_value = 0
        if state in self.blocked:   # Skip blocked states
          continue

        # Current coordinate
        curr_coord = self.to_coord(state)

        # Action from policy
        action = np.random.choice(self.policy[state], size=1)[0]
        action = np.random.randint(0,4) if isinstance(action, str) else action

        # Check all possible next states
        for action_next in self.actions:
          
          # (1 - noise) if correct next state
          if action_next == action:
            prob = 1 - self.noise

          # noise / 2 if adjacent next state
          elif action_next != (action + 2) % 4:
            prob = self.noise / 2
          
          # 0 if opposite next state
          else:
            prob = 0

          state_next, reward, done, info = self.step(action_next, noise=0, state=curr_coord)
          state_next = self.to_state(state_next)
          
          # Get sum 
          curr_value += prob * (reward + self.gamma * self.value[state_next])
          
        # Update max value across all states
        delta = max(delta, abs(curr_value - new_value[state]))

        # Update current state
        new_value[state] = curr_value

      # Replace variable
      self.value = new_value

      # Return if threshold is reached
      if delta < tolerance or it == max_iter - 1:
        break

  # Policy Improvement
  def PolicyImprovement(self):
    self.policy = [['.'] for _ in range(self.N)]
    self.Q = [[0 for _ in range(len(self.actions))] for _ in range(self.N)]

    # Check all states
    for state in range(self.N):
      if state in self.blocked + self.nterminal:   # Skip blocked states
        continue

      # Current coordinate
      curr_coord = self.to_coord(state)

      # Run through all actions
      for action in self.actions:

        # Check all possible next states
        for action_next in self.actions:
          
          # (1 - noise) if correct next state
          if action_next == action:
            prob = 1 - self.noise

          # noise / 2 if adjacent next state
          elif action_next != (action + 2) % 4:
            prob = self.noise / 2
          
          # 0 if opposite next state
          else:
            prob = 0

          state_next, reward, done, info = self.step(action_next, noise=0, state=curr_coord)
          state_next = self.to_state(state_next)
          
          # Get sum 
          self.Q[state][action] += prob * (reward + self.gamma * self.value[state_next])

      # Get optimal policy
      self.policy[state] = list(np.flatnonzero(self.Q[state] == np.max(self.Q[state])))
    
  # Policy Iteration to determine optimal policy
  def PolicyIteration(self, max_iter = 100, tolerance = 1e-5, show_iter = False):
    self.value = [0 for _ in range(2*self.N)]
    self.policy = [['.'] for _ in range(self.N)]

    for it in range(max_iter):
        old_policy = self.policy.copy()
        self.PolicyEvaluation(max_iter, tolerance)
        self.PolicyImprovement()
        
        # Break if policy did not change
        if self.policy == old_policy or it == max_iter - 1:
          print("Final")
          self.show_current(policy=True)
          break

        # Display iteration
        if show_iter:
          print(f"Iteration: {it+1}")
          self.show_current()

  # Convert action to arrows
  def to_arrow(self, action):
    if action in self.convert:
      return self.convert[action]
    return "." 
  
  # Print the current value function and/or policy
  def show_current(self, value=True, policy=False):    
    if value:
      print("Value Function")
      for y in range(self.size[1]-1,-1,-1):
        out = ""
        for x in range(self.size[0]):
          out += str(round(self.value[self.to_state([x,y,0])],2)) + "\t"
        print(out)

    if policy:
      print("\nCurrent Policy")
      for y in range(self.size[1]-1,-1,-1):
        out = ""
        for x in range(self.size[0]):
          out += self.to_arrow(self.policy[self.to_state([x,y,0])][0]) + "\t"
        print(out)
    print('- '*20 + '\n')

  # Add grids to render
  def add_grids(self, canvas):
    for x in range(0, self.size[0]):
      for y in range(0, self.size[1]):
        canvas[y*self.scale, 0:self.size[0]*self.scale] = 0
        canvas[0:self.size[1]*self.scale, x*self.scale] = 0
    return canvas
  
  # Overlay agent
  def overlay_agent(self):
    x = self.state[0]; y = self.size[1] - self.state[1]
    new_canvas = self.canvas.copy()
    if self.state[2] == 1:
      self.oops = "Goal State reached"
    for nx in range(x*self.scale, x*self.scale+self.scale):
      for ny in range(y*self.scale-self.scale, y*self.scale):
        if np.sum(self.agent[ny - y*self.scale, nx - x*self.scale]) > 2.9:
          new_canvas[ny, nx] = self.canvas[ny, nx]
        else:
          new_canvas[ny, nx] = self.agent[ny - y*self.scale, nx - x*self.scale]
    return new_canvas
    
  # Other errors
  def overlay_errors(self, canvas):
    x = self.state[0]; y = self.size[1] - self.state[1]

    # Add bumped area
    x_range = range(0); y_range = range(0)
    if self.bumped == 0:
      x_range = range(x*self.scale, x*self.scale+self.scale)
      y_range = range(y*self.scale-self.scale, int((y-0.95)*self.scale))
    if self.bumped == 1:
      x_range = range(int((x+0.95)*self.scale), x*self.scale+self.scale)
      y_range = range(y*self.scale-self.scale, y*self.scale)
    if self.bumped == 2:
      x_range = range(x*self.scale, x*self.scale+self.scale)
      y_range = range(int((y+0.95)*self.scale)-self.scale, y*self.scale)
    if self.bumped == 3:
      x_range = range(x*self.scale, int((x-0.95)*self.scale+self.scale))
      y_range = range(y*self.scale-self.scale, y*self.scale)

    for nx in x_range:
      for ny in y_range: 
        canvas[ny, nx] = [226/255,43/255,138/255]
    
    # Add noise 
    pos = (0, int(self.scale*0.2))
    cv2.putText(canvas, self.oops, pos, cv2.FONT_HERSHEY_DUPLEX, fontScale=0.5, color=0)

    return canvas
    
  # Render the environment and agent
  def render(self, mode = "human", wait=800):
    new_canvas = self.overlay_agent()
    new_canvas = self.add_grids(new_canvas)
    new_canvas = self.overlay_errors(new_canvas)
    if mode == "human":
      cv2.imshow("GridWorld", new_canvas)
      cv2.waitKey(wait) 
    else:
      return new_canvas
  
  # Reset the environment and agent
  def reset(self):
    self.state = self.start
    self.bumped = -1
    self.oops = ""
    self.value = [0 for _ in range(2*self.N)]
    self.policy = [['.'] for _ in range(self.N)]
    return self.state
  
  # Close all render windows
  def close(self):
    cv2.destroyAllWindows()

## Model-Free Reinforcement Learning

The functions below trains an agent in the environment.

### Create Reinforcement Learning Model

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [4]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=states))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(40, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

### Build Agent

In [5]:
from rl.agents import DQNAgent, SARSAAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [6]:
def build_agent(model, actions):
    memory = SequentialMemory(limit=50000, window_length=1)
    policy = BoltzmannQPolicy()
    agent = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    # policy = EpsGreedyQPolicy()
    # agent = SARSAAgent(model=model, policy=policy, nb_actions=actions, nb_steps_warmup=10)
    return agent

### Train Agent

In [7]:
def train_agent(env):
    states = env.observation_space.shape
    actions = env.action_space.n
    model = build_model(states, actions)
    agent = build_agent(model, actions)
    agent.compile(Adam(learning_rate=1e-2), metrics=['mae'])
    agent.fit(env, nb_steps=10000, visualize=False, verbose=1)
    return agent

## Example

The example below is based on the following environment.

<img src="assets/Cliff.png" width="300">

In [8]:
env = GridWorld([5,5], start=[0,1,0], reward=0)
env.add_blocked([1,2,0])
env.add_blocked([1,3,0])
env.add_blocked([3,2,0])
env.add_nterminal([0,0,0], -10)
env.add_nterminal([1,0,0], -10)
env.add_nterminal([2,0,0], -10)
env.add_nterminal([3,0,0], -10)
env.add_nterminal([4,0,0], -10)
env.add_nterminal([2,2,0], 1)
env.add_nterminal([4,2,0], 10)

### Sample Simulation

The cell below accepts a string containing the direction of the agent and renders a sample simulation.

In [9]:
to_int = {'u': 0, 'r': 1, 'd': 2, 'l': 3}

n_state = env.reset()
done = False
score = 0

actions = input('Input Move List: ')
# Example: uuurrrrddd

env.render()
for action in actions:
    if action in to_int:
        action = to_int[action]
    else:
        print('Please input a valid action')
        env.close() 
        break
    n_state, reward, done, info = env.step(action)
    score += reward
    env.render()
    if done: 
        break
print(f'Score: {score}')
env.close()

Score: 1


### Model-Based Case

This subsection presents the optimal policy using `ValueIteration` and `PolicyIteration`.

In [10]:
env.ValueIteration(show_iter=False, max_iter=100)

Final
Value Function
4.48	5.17	5.88	6.68	7.51	
3.93	0	6.03	7.51	8.65	
3.45	0	1.0	0	10.0	
2.93	2.0	3.31	5.72	8.48	
-10.0	-10.0	-10.0	-10.0	-10.0	

Current Policy
→	→	→	→	↓	
↑	.	→	→	↓	
↑	.	.	.	.	
↑	↑	→	→	↑	
.	.	.	.	.	
- - - - - - - - - - - - - - - - - - - - 



In [11]:
env.PolicyIteration(show_iter=False, max_iter=100)

Final
Value Function
4.48	5.17	5.88	6.68	7.51	
3.93	0	6.03	7.51	8.65	
3.45	0	1.0	0	10.0	
2.93	2.0	3.31	5.72	8.48	
-10.0	-10.0	-10.0	-10.0	-10.0	

Current Policy
→	→	→	→	↓	
↑	.	→	→	↓	
↑	.	.	.	.	
↑	↑	→	→	↑	
.	.	.	.	.	
- - - - - - - - - - - - - - - - - - - - 



### Model-Free Case

This subsection determines the optimal policy using the previously defined functions.

Note: Weird behavior can be observed in the model-free case. This might be due to lack of iterations and/or the type of policy/agent used.

In [12]:
agent = train_agent(env)

Training for 10000 steps ...
Interval 1 (0 steps performed)


C:\Users\brian\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


10000/10000 [==============================] - 187s 19ms/step - reward: -0.0141
done, took 187.317 seconds


In [13]:
scores = agent.test(env, nb_episodes=1, visualize=True)
env.close()

Testing for 1 episodes ...
Episode 1: reward: 10.000, steps: 46
